Simple Gen AI APP using LangChain

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
#langsmith tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
# Data Ingestion=From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader


In [9]:
loader=WebBaseLoader("https://www.geeksforgeeks.org/introduction-to-langchain/")
loader

In [10]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIntroduction to LangChain - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\nCoursesDSA to DevelopmentFor Working ProfessionalsData Structure & Algorithm Classes (Live)System Design (Live)JAVA Backend Development(Live)DevOps(Live)Data Structures & Algorithms in PythonFor StudentsInterview Preparation CourseGATE CS & IT 2024Data Science (Live)Data Structure & Algorithm-Self Paced(C++/JAVA)Master Competiti

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [12]:
documents

[Document(metadata={'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US'}, page_content='Introduction to LangChain - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content'),
 Document(metadata={'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US'}, page_content='

In [13]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [14]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

NameError: name 'embeddings' is not defined

In [ ]:
vectorstoredb

In [ ]:
#Query from a vector db
query="Code analysis: LangChain can be used to"
result=vectorstoredb.similarity_search(query)
result[0].page_content

In [15]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
#Retrieval chain, Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Code analysis: LangChain can be used to",
    "context":[Document(page_content="Code analysis: LangChain can be used to analyse code and find potential bugs or security flaws.")]

})

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question .

In [ ]:
#Input-->Retriever-->vectorstoredb
vectorstoredb

In [17]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

NameError: name 'vectorstoredb' is not defined

In [ ]:
retrieval_chain

In [ ]:
#get the response form the LLM
response=retrieval_chain.invoke({"input":"Code analysis: LangChain can be used to"})
response["answer"]

In [ ]:
response

In [ ]:
response["context"]